<h2>Slitmask spectroscopic reduction: KOSMOS College of Idaho

Notebook goes through basic ideas of reducing KOSMOS slitmask spectra. Does not discuss basic reduction (see KOSMOS notebook) but concentrates on multi-slit issues: finding the slitlets, doing wavelength calibration for each slitlet (trickiest part), 2D extraction of slitlits, including wavelength correction by skylines and undistorting along slit to central wavelength, and simple 1D extraction.

In [1]:
from pyvista import imred, tv, stars, slitmask, image, spectra
import pdb
import copy
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.table import vstack

In [2]:
# you may need/want to use qt or osx in the next line
%matplotlib tk    
t=tv.TV()

In [3]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/red/UT230909',verbose=False)                                                         
flat = red.reduce(21)
arcs= red.sum([24])

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Find slit edges from flat, and fit polynomials to locations

In [4]:
trace=spectra.Trace(transpose=True)
t.tvclear()
bottom,top = trace.findslits(flat,display=t,thresh=0.5,sn=True)

Plot shape of slit edges relative to center position, just for information

In [5]:
x=np.arange(4096)
plt.figure()
for l in bottom :
    plt.plot(x,l(x)-l(2048))
    plt.xlabel('x')
    plt.ylabel('y-y(2048)')

Read in the slitmask file, which we'll use to get object names and slit locations to help with wavelength solution.

If the number of slitlets found doesn't match the number of targets, you'll need to go back and adjust the threshold to find the correct slitlets, or else modify the targets table below so that they match.

In [6]:
kmsfile='kosmos.23.seg3g2.kms'
targets=slitmask.read_kms(kmsfile,sort='YMM')
if len(targets) != len(bottom) : 
    print('ERROR, number of identified slits does not match number of targets')
targets

ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str2,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG113,NN,STRAIGHT,4.0,4.0,0.0,212134.279,191220.25,0.683,0.683,-5.252,-34.315
TARG112,NN,STRAIGHT,4.0,4.0,0.0,212140.986,191141.81,0.683,0.683,-21.469,-27.768
TARG114,NN,STRAIGHT,4.0,4.0,0.0,212123.429,191104.21,0.683,0.683,20.979,-21.323
TARG111,NN,STRAIGHT,0.9,10.0,0.0,212132.879,191002.48,0.154,1.707,-1.87,-10.801
TARG110,NN,STRAIGHT,0.9,10.0,0.0,212127.784,190908.67,0.154,1.707,10.451,-1.61
TARG109,NN,STRAIGHT,0.9,10.0,0.0,212128.763,190845.78,0.154,1.707,8.085,2.297
TARG108,NN,STRAIGHT,0.9,10.0,0.0,212127.508,190828.88,0.154,1.707,11.119,5.182
TARG107,NN,STRAIGHT,0.9,10.0,0.0,212133.847,190811.86,0.154,1.707,-4.212,8.077
TARG115,NN,STRAIGHT,4.0,4.0,0.0,212132.347,190735.58,0.683,0.683,-0.582,14.272


In [7]:
# if you want subset of slitlets, e.g. ignoring alignments stars, select them here
#gd=np.where(targets['NAME']!='NN')[0]     # or, e.g., gd=[2,3,4,7,9]
gd=[3,4,5,6,7,9,10,11,13,15]
gdtrace=copy.deepcopy(trace)
gdtrace.model = [trace.model[i] for i in gd]
gdtrace.rows = [trace.rows[i] for i in gd]
trace=copy.deepcopy(gdtrace)
targets=targets[gd]
print(targets)
vars(trace)

   ID   NAME  SHAPE   WID LEN  ROT   ALPHA      DELTA   WIDMM LENMM   XMM     YMM  
------- ---- -------- --- ---- --- ---------- --------- ----- ----- ------- -------
TARG111   NN STRAIGHT 0.9 10.0 0.0 212132.879 191002.48 0.154 1.707   -1.87 -10.801
TARG110   NN STRAIGHT 0.9 10.0 0.0 212127.784 190908.67 0.154 1.707  10.451   -1.61
TARG109   NN STRAIGHT 0.9 10.0 0.0 212128.763 190845.78 0.154 1.707   8.085   2.297
TARG108   NN STRAIGHT 0.9 10.0 0.0 212127.508 190828.88 0.154 1.707  11.119   5.182
TARG107   NN STRAIGHT 0.9 10.0 0.0 212133.847 190811.86 0.154 1.707  -4.212   8.077
TARG106   NN STRAIGHT 0.9 10.0 0.0 212133.934 190720.87 0.154 1.707  -4.422   16.78
TARG105   NN STRAIGHT 0.9 10.0 0.0 212130.054 190659.25 0.154 1.707   4.962  20.475
TARG104   NN STRAIGHT 0.9 10.0 0.0 212136.755 190631.54 0.154 1.707 -11.247  25.194
TARG102   NN STRAIGHT 0.9 10.0 0.0 212134.103 190555.71 0.154 1.707  -4.831  31.313
TARG101   NN STRAIGHT 0.9 10.0 0.0 212133.821 190534.28 0.154 1.707   -4.15 

{'type': 'Polynomial1D',
 'degree': 2,
 'sigdegree': 0,
 'pix0': 0,
 'spectrum': None,
 'rad': 5,
 'transpose': True,
 'lags': range(-50, 50),
 'model': [Polynomial([ 725.36544175,    6.39378253,   -3.62030736], domain=[   98.,  3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([ 934.6344652 ,    6.49094214,    1.56190524], domain=[   98.,  3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([ 1023.38565965,     6.46450255,     3.67533735], domain=[   98.,  3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([ 1089.05958351,     6.49312832,     5.26758998], domain=[   98.,  3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([ 1154.49886865,     6.51239823,     6.81155863], domain=[   98.,  3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([ 1352.12850914,     6.53479617,    11.59481407], domain=[   98.,  3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([ 1436.10006515,     6.52573701,    13.55642517], domain=[   98.,  3998.], window=[-1.,  1.], symbol='x'),
  Polynomial([

Using the derived traces, extract the slitlets for arcs

In [8]:
arcec=trace.extract2d(arcs,display=t)

extracting: 
 725-764
 934-973
 1023-1062
 1089-1128
 1154-1193
 1352-1391
 1436-1475
 1542-1581
 1681-1720
 1763-1802
  See extraction window(s). Hit space bar to continue....


Add XMM and YMM for each slit to headers of each extracted image. They need to match so you get the right values!

In [9]:
for arc,target in zip(arcec,targets) : 
    arc.header['XMM'] = target['XMM']
    arc.header['YMM'] = target['YMM']

Now loop through each extracted arc to do wavelength calibration. This requires a little effort because the change in the location of the slit relative to the default saved wavelength calibration is significant enough that it can be a challenge to automatically find the lines, since the change in spectrum is more than a simple shift (and, in fact, more than a shift + dispersion change). 

However, a simple shift is usually enough to identify some of the lines, and these can be used to bootstrap the wavelength solution; the initial identification is easier if you use an estimate of the shift from the mask design XMM.

You can use identify() to do the iteration. On the first pass, only central lines may be correctly identified. Use 'l' and 'r' to remove lines to the left and right of the identified lines. Then use 'i' to iterate, i.e., allow it to re-identify lines (i just returns True to allow you to iterate). When happy with solution, use ' ' to move onto the final 2D wavelength calibration.

You can really help this process if you supply an initial wavelength calibration (a pyvista WaveCal object) that was done using the same lamp(s) as your arc exposures (here, using KOSMOS_red_waves.fits'), and using a master line list that corresponds to these lamp(s) (here, using ne.dat). If you choose another WaveCal to start from, you may need to get a correct approximate relation for the shift from the reference spectrum as a function of XMM. For KOSMOS, seems like -22.5(XMM) gives a rough pixel shift from a center slit location, -22.5(XMM-24.44) for a low slit location.

The next cell just shows how the simple shift fails, even using the XMM for each slit to shift: all of the arc lines don't match up with just a translation.

In [10]:
plt.figure()
wav2=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
plt.plot(wav2.spectrum[0])
for i, arc in enumerate(arcec[0:1]) :
    shift=(arc.header['XMM']*-22.5)
    plt.plot(arc.data[19][int(shift):]*30)
    print(shift)
    plt.draw()

  rms:    0.177 Angstroms (50 lines)
42.075


The wavelength calibration for each slitlet will be saved. Since this is probably the most time-consuming part, you could use saved one if you only want to redo a couple of them (sometimes, you hit the wrong key and want to do one over!)

In [17]:
clobber=False    # set to False if you want to use any saved ones
for i,(arc,targ) in enumerate(zip(arcec,targets)) :
    wavname = 'CofIwav_{:s}.fits'.format(targ['ID'])
    if clobber or not os.path.exists(wavname) :
        wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
        wav.fit(degree=3)
        nrow=arc.shape[0]
    
        # get initial guess at shift from reference using XMM (KOSMOD red low!)
        shift=int(arc.header['XMM']*-22.5) # +550  #-wav.pix0)
        lags=np.arange(shift-20,shift+20)

        iter = True
        while iter :
            iter = wav.identify(arc[nrow//2],plot=True,plotinter=True,lags=lags,thresh=10,file='./copy_new_neon_red_center.dat')
            lags=np.arange(-50,50)
            plt.close()

        bd= np.where(wav.weights<0.5)[0]
        print(wav.waves[bd])
        # Do the 2D wavelength solution, sampling 10 locations across slitlet
        wav.degree=5
        wav.identify(arc,plot=True,nskip=nrow//10,thresh=10)
        plt.close()
        wav.write(wavname)
        wav.add_wave(arc)
        t.tv(wav.correct(arc,arc.wave[nrow//2]))

  rms:    0.177 Angstroms (50 lines)
  rms:    0.177 Angstroms (50 lines)
  cross correlating with reference spectrum using lags between:  88 127
  Derived pixel shift from input wcal:  [ 122.23720685]
  See identified lines.
  rms:  129.441 Angstroms (34 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       i : return with True value (to allow iteration)
       anything else : finish and return
  rms:  129.441 Anstroms
  input from plot window...

[]
  cross correlating with reference spectrum using lags between:  -300 299
  Derived pixel shift from input wcal for row: 38 1
  See identified lines.
  rms:   41.698
rejecting 1 points from 225 total: 
  rms:    3.692
rejecting 1 points from 225 total: 
  See 2D wavecal fit. Enter space in plot window to continue



ValueError: The Axes must have been created in the present figure

Now set up routine to reduct/extract science frames

In [ ]:
def multi_extract2d(red,trace,targets,image,bias=None,dark=None,flat=None,display=None,crbox='lacosmic',crsig=10,
                    rad=5) :

    if display is not None : 
        display.clear()
        plot = True
    else:
        plot = False
        
    # basic image read and reduction
    imcr=red.reduce(image,bias=bias,dark=dark,flat=flat,crbox=crbox,display=display,crsig=crsig)

    # 2D extraction
    out = trace.extract2d(imcr,display=display)

    # loop over each desired slitlet, get wavelength shift from skylines, undistort to central wavelengths
    diff=[]
    for i,(o,targ) in enumerate(zip(out,targets)) :
        wav=spectra.WaveCal('./CofIwav_{:s}.fits'.format(targ['ID']))
        orig=wav.model.c0_0
        wav.add_wave(o)

        # set rows to use for skyline
        nrows = o.shape[0]
        rows = [x for x in range(0,nrows) if np.abs(x-nrows//2)>rad]
        wav.skyline(o,thresh=10,rows=rows,plot=plot)
        
        if plot : plt.close()
        wav.add_wave(o)
        if display is not None : display.tv(o)
        out[i]=wav.correct(o,o.wave[nrows//2])
        if display is not None : display.tv(out[i])
        name = out[i].header["FILE"].split(".")[0] 
        out[i].write('{:s}_{:s}_2d.fits'.format(name,targ['ID']))
        diff.append(wav.model.c0_0-orig)

    print('shifts: ',diff)
    return out


In [18]:
out=multi_extract2d(red,trace,targets,20,display=None)

  starting CR rejection, may take some time ....
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
extracting: 
 725-764
 934-973
 1023-1062
 1089-1128
 1154-1193
 1352-1391
 1436-1475
 1542-1581
 1681-1720
 1763-1802
  rms:    0.031
rejecting 7 points from 479 total: 
  rms:    0.031
rejecting 7 points from 479 total: 
  rms:    0.151
rejecting 1 points from 203 total: 
  rms:    0.146
rejecting 1 points from 203 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.039
rejecting 22 points from 490 total: 
  rms:    0.039
rejecting 22 points from 490 total: 
  rms:    0.095
rejecting 1 points from 200 total: 
  rms:    0.092
rejecting 1 points from 200 total: 

appending uncertainty
appending bitmask
appending wave
  rms:    0.055
rejecting 1 points from 493 total: 
  rms:    0.055
rejecting 1 points from 493 total: 
  rms:    0.129
rejecting 0 points from 186 total: 

appending uncertainty
appending bitmask
append

/home/local/Anaconda3-2020.02/envs/pyvista/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


  rms:    3.500
rejecting 1 points from 86 total: 
  rms:    3.210
rejecting 1 points from 86 total: 

appending uncertainty
appending bitmask
appending wave
  rms:   23.703
rejecting 4 points from 246 total: 
  rms:   23.703
rejecting 4 points from 246 total: 
  rms:    0.718
rejecting 0 points from 50 total: 

appending uncertainty
appending bitmask
appending wave
shifts:  [-0.12296180385965272, -0.093775605931114114, -0.12852487187319639, -0.079896214060681814, -0.044637526895712654, -0.05700723991230916, -0.11309212074411334, -0.072314127542085771, -3.0352685421512433, -0.69899470692053001]


As desired, move on to 1D extraction

In [19]:
def multi_extract1d(spec2d) :
    def model(x) :
        return x*0.

    fig=plt.figure()
    spec1d=[]
    for i in range(len(spec2d)) :
        trace1 = spectra.Trace(transpose=False)
        trace1.rows = [0,spec2d[i].data.shape[0]]
        trace1.index = [0]
        peak,ind = trace1.findpeak(spec2d[i],thresh=10,sort=True)
        if len(peak) > 0:
            def model(x) :
                return x*0. + peak[0]
            trace1.model = [model]
            spec=trace1.extract(spec2d[i],rad=4,back=[[-10,-5],[5,10]],display=None)
            plt.figure(fig)
            spec.wave = out[i].wave[peak]
            print(spec.wave[0].shape,spec.data[0].shape)

            plt.subplot(2,1,1)
            plt.plot(spec.wave[0],spec.data[0])
            plt.subplot(2,1,2)
            plt.plot(spec.wave[0],spec.sky[0])
            spec1d.append(spec)
        else :
            print('no peak found for slit: ',i)
    plt.tight_layout()
    return spec1d

In [ ]:
spec1d = multi_extract1d(out)

looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [20, 6]
aperture/fiber:  [1, 0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [20, 6]
aperture/fiber:  [1, 0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [18, 5]
aperture/fiber:  [1, 0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [20, 6]
aperture/fiber:  [1, 0]
  extracting ... 

(4096,) (4096,)
looking for peaks using 200 pixels around 2048, threshhold of 10.000000
peaks:  [18]
aperture/fiber:  [0]
  extracting ... 


In [39]:
spec1d[0].skyerr


array([[25.51282701, 29.67282122, 28.87833749, ..., 27.52643757,
        29.13018955, 28.69354657]])